## Einleitung

Dieses Notebook führt statistische Tests und Analysen auf den PRECISION-Daten durch.

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency, ttest_ind, mannwhitneyu, f_oneway
import warnings
warnings.filterwarnings('ignore')

# Plotting-Einstellungen
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

# PRECISION Farben
COLORS = ['#1a365d', '#3182ce', '#2c5282', '#38a169', '#d69e2e']

print('Setup erfolgreich!')

## 1. Daten laden

In [ ]:
# Daten laden
df = pd.read_csv('../data/sample/patient_data.csv')
print(f'Datensatz geladen: {len(df)} Zeilen, {len(df.columns)} Spalten')
df.head()

## 2. Deskriptive Statistik nach Gruppen

In [ ]:
# Zusammenfassung nach Behandlungsgruppe
numeric_cols = ['age', 'bmi', 'blood_pressure_sys', 'blood_pressure_dia', 'cholesterol', 'glucose']

summary_by_group = df.groupby('treatment_group')[numeric_cols].agg(['mean', 'std', 'median']).round(2)
summary_by_group

In [ ]:
# Response-Rate pro Gruppe
response_summary = df.groupby('treatment_group').agg(
    n_total=('patient_id', 'count'),
    n_responders=('response', 'sum'),
    response_rate=('response', 'mean')
).round(3)

response_summary['response_rate_pct'] = (response_summary['response_rate'] * 100).round(1)
response_summary

## 3. Statistische Tests

### 3.1 Chi-Quadrat-Test: Response vs. Behandlungsgruppe

In [ ]:
# Kontingenztabelle erstellen
contingency_table = pd.crosstab(df['treatment_group'], df['response'])
print('Kontingenztabelle:')
print(contingency_table)
print()

# Chi-Quadrat-Test
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

print('Chi-Quadrat-Test Ergebnisse:')
print(f'  Chi² = {chi2:.4f}')
print(f'  p-Wert = {p_value:.4f}')
print(f'  Freiheitsgrade = {dof}')
print()

if p_value < 0.05:
    print('→ Signifikanter Zusammenhang zwischen Behandlungsgruppe und Response (p < 0.05)')
else:
    print('→ Kein signifikanter Zusammenhang gefunden (p >= 0.05)')

### 3.2 ANOVA: Alter nach Behandlungsgruppe

In [ ]:
# ANOVA für Alter zwischen Gruppen
groups = [group['age'].values for name, group in df.groupby('treatment_group')]
f_stat, p_value_anova = f_oneway(*groups)

print('ANOVA: Alter nach Behandlungsgruppe')
print(f'  F-Statistik = {f_stat:.4f}')
print(f'  p-Wert = {p_value_anova:.4f}')
print()

if p_value_anova < 0.05:
    print('→ Signifikante Unterschiede im Alter zwischen den Gruppen')
else:
    print('→ Keine signifikanten Altersunterschiede zwischen den Gruppen (gut für Randomisierung!)')

### 3.3 T-Test: Responder vs. Non-Responder

In [ ]:
# T-Tests für verschiedene Variablen
responders = df[df['response'] == 1]
non_responders = df[df['response'] == 0]

test_results = []

for col in numeric_cols:
    t_stat, p_val = ttest_ind(responders[col], non_responders[col])
    
    test_results.append({
        'Variable': col,
        'Mean Responder': responders[col].mean(),
        'Mean Non-Responder': non_responders[col].mean(),
        't-Statistik': t_stat,
        'p-Wert': p_val,
        'Signifikant': 'Ja' if p_val < 0.05 else 'Nein'
    })

results_df = pd.DataFrame(test_results).round(4)
results_df

## 4. Korrelationsanalyse

In [ ]:
# Korrelationsmatrix
corr_matrix = df[numeric_cols].corr()

# Heatmap
fig, ax = plt.subplots(figsize=(10, 8))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)

sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, square=True, linewidths=0.5, ax=ax,
            cbar_kws={'label': 'Korrelationskoeffizient'})

ax.set_title('Korrelationsmatrix der numerischen Variablen', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../assets/images/correlation_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Logistische Regression (Einfach)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import cross_val_score

# Features vorbereiten
X = df[numeric_cols].copy()
y = df['response']

# Standardisierung
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Modell trainieren
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_scaled, y)

# Cross-Validation
cv_scores = cross_val_score(model, X_scaled, y, cv=5, scoring='accuracy')

print('Logistische Regression - Ergebnisse')
print('=' * 40)
print(f'Cross-Validation Accuracy: {cv_scores.mean():.3f} (+/- {cv_scores.std()*2:.3f})')
print()

# Koeffizienten
coef_df = pd.DataFrame({
    'Variable': numeric_cols,
    'Koeffizient': model.coef_[0],
    'Odds Ratio': np.exp(model.coef_[0])
}).sort_values('Koeffizient', key=abs, ascending=False)

print('Feature-Wichtigkeit (nach Koeffizient):')
coef_df.round(4)

In [ ]:
# Feature Importance Plot
fig, ax = plt.subplots(figsize=(10, 6))

colors = [COLORS[0] if c > 0 else COLORS[4] for c in coef_df['Koeffizient']]
bars = ax.barh(coef_df['Variable'], coef_df['Koeffizient'], color=colors, alpha=0.8)

ax.axvline(x=0, color='gray', linestyle='--', linewidth=1)
ax.set_xlabel('Koeffizient (standardisiert)')
ax.set_title('Feature-Wichtigkeit für Response-Vorhersage', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('../assets/images/feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Ergebnisse exportieren

In [ ]:
# Alle Ergebnisse in ein Dictionary für den Report
analysis_results = {
    'chi2_test': {
        'chi2': chi2,
        'p_value': p_value,
        'significant': p_value < 0.05
    },
    'anova_age': {
        'f_stat': f_stat,
        'p_value': p_value_anova,
        'significant': p_value_anova < 0.05
    },
    'logistic_regression': {
        'cv_accuracy_mean': cv_scores.mean(),
        'cv_accuracy_std': cv_scores.std()
    }
}

# Als JSON speichern
import json
with open('../data/sample/analysis_results.json', 'w') as f:
    json.dump(analysis_results, f, indent=2)

print('Ergebnisse gespeichert in: data/sample/analysis_results.json')

## 7. Zusammenfassung

### Statistische Erkenntnisse:

1. **Randomisierung erfolgreich:** Keine signifikanten Altersunterschiede zwischen Behandlungsgruppen
2. **Behandlungseffekt:** Chi²-Test zeigt [Ergebnis abhängig von Daten]
3. **Prädiktoren:** Wichtigste Variablen für Response-Vorhersage identifiziert
4. **Modell-Performance:** Logistische Regression erreicht ~[X]% Accuracy